In [57]:
import csv
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import math
from matplotlib.colors import ListedColormap
import os
from scipy.signal import savgol_filter
import re

In [ ]:
def params_calc(df):
    num_columns = df_csv.shape[1]
    num_cycles = int((num_columns-2)/2)
    num_svar = df_csv[0].nunique()
    num_fvar = int(df_csv.shape[0]/num_svar)
    data = np.ndarray((num_columns, num_fvar, num_svar), dtype = 'float')
    max_currents = np.zeros((num_svar))
    min_currents = np.zeros((num_svar))
    
    for k in range(0,num_svar):
        for i in range(0,num_columns):
            for j in range(0,num_fvar):
                data[i,j,k] = df.iloc[j+num_fvar*k,i]

In [206]:
def whole_calc(filepath):
    
    device_coords = []
    for root, dirs, files in os.walk(filepath):
        for file in files:
            some_arr = []
            if os.path.splitext(file)[1] == '.dat':
                device_coords.append(np.array([int(s) for s in re.split('_', os.path.splitext(file)[0]) if s.isdigit()]))
                f_dat = open(root + '\\' + file, 'r')
                lines = f_dat.readlines()
                for line in lines:
                    if 'l' in line:
                        some_arr.append(lines.index(line))
                for line in lines:
                    if line.startswith('0,00000E+0'):
                        some_arr.append(lines.index(line))
                some_arr = np.array(some_arr)
                for item in some_arr:
                    lines.pop(item)
                    some_arr -= 1
                f_csv = open(root + '\\' + os.path.splitext(file)[0] + '.csv', 'w')
                for line in lines:
                    line = line.replace(',', '.')
                    f_csv.write(line) 
                f_dat.close()
                f_csv.close()
    
    all_devices = np.array(device_coords[0])
    for i in range(1, len(device_coords)):
        all_devices = np.vstack((all_devices, device_coords[i]))
    
    open_state_resistance = np.zeros((len(np.unique(all_devices[:,0])), len(np.unique(all_devices[:, 1]))))
    closed_state_resistance = np.zeros((len(np.unique(all_devices[:,0])), len(np.unique(all_devices[:, 1]))))
    on_off_ratio = np.zeros((len(np.unique(all_devices[:,0])), len(np.unique(all_devices[:, 1]))))
    
    try:
        os.makedirs(filepath + '\\On/off')
        os.makedirs(filepath + '\\Open state resistance')
        os.makedirs(filepath + '\\Closed state resistance')
        os.makedirs(filepath + '\\Gate resistance')
        os.makedirs(filepath + '\\Log scale graphs')
    except:
        pass
    for root, dirs, files in os.walk(filepath):
        for file in files:
            if os.path.splitext(file)[1] == '.csv':
                df_csv = pd.read_csv(root + '\\' + file, engine = 'python', sep = '	', header = None)
                num_columns = df_csv.shape[1]
                num_cycles = (num_columns-2)/2
                num_svar = df_csv[0].nunique()
                num_fvar = df_csv.shape[0]/num_svar

In [207]:
whole_calc(r'C:\Users\kpebe\OneDrive\Рабочий стол\Transistors\\For new program')

12